# Assignment: Direct Stiffness Method (DSM) for 2D Trusses

**Goal:** Implement and verify a complete Direct Stiffness Method solver for planar trusses:

- Assemble the global stiffness matrix from truss elements  
- Apply boundary conditions  
- Solve for nodal displacements  
- Recover element axial forces / stresses and nodal reactions  
- Validate your implementation using built-in tests and benchmark examples  

---

## Learning outcomes
By the end you should be able to:
1. Derive and implement the 2D truss element stiffness in global coordinates.
2. Assemble a global stiffness matrix using element connectivity.
3. Apply essential boundary conditions (fixed DOFs) robustly.
4. Post-process internal forces and reactions.
5. Verify correctness with equilibrium/energy checks and known solutions.

---

## What you will submit
1. This notebook with all **TODOs** completed.
2. A short write-up (1–2 pages) answering the questions in **Part E**.
3. (Optional) Plots of undeformed/deformed geometry for the benchmark trusses.

---

## Conventions (IMPORTANT)
- Nodes are **0-indexed**
- Each node has 2 DOFs: `(ux, uy)`
- DOF mapping: `dof = 2*node + {0 for ux, 1 for uy}`


## Setup

In [ ]:
import numpy as np
np.set_printoptions(precision=6, suppress=True)


# Part A — Core DSM implementation (TODOs)

Implement the following functions:

- `element_stiffness_global_2d_truss(...)`
- `assemble_global_stiffness(...)`
- `apply_boundary_conditions_by_partition(...)`
- `solve_truss(...)`
- `recover_element_axial_forces(...)`

Do **not** change the function signatures.


In [ ]:
def element_stiffness_global_2d_truss(xi, yi, xj, yj, E, A):
    """
    Returns the 4x4 truss element stiffness matrix in GLOBAL coordinates.

    Parameters
    ----------
    (xi, yi), (xj, yj) : float
        Node coordinates.
    E : float
        Young's modulus.
    A : float
        Cross-sectional area.

    Returns
    -------
    ke : (4,4) ndarray
        Element stiffness in global coordinates for DOFs [ui_x, ui_y, uj_x, uj_y].
    L : float
        Element length.
    c : float
        cos(theta)
    s : float
        sin(theta)
    """
    # TODO: compute L, c, s and assemble ke
    # Hint: ke = (EA/L) * [[ c^2,  cs, -c^2, -cs],
    #                      [ cs,  s^2, -cs,  -s^2],
    #                      [-c^2, -cs,  c^2,  cs],
    #                      [-cs, -s^2,  cs,  s^2]]
    raise NotImplementedError


def assemble_global_stiffness(nodes, elements):
    """
    Assemble the global stiffness matrix K for a 2D truss.

    Parameters
    ----------
    nodes : (n,2) ndarray
        Node coordinates.
    elements : list of dict
        Each element dict must have:
        - 'i', 'j' : node indices
        - 'E', 'A' : properties (floats)

    Returns
    -------
    K : (2n, 2n) ndarray
        Global stiffness matrix.
    elem_data : list of dict
        Per-element cached data (e.g., L,c,s, ke) helpful for postprocessing.
    """
    n = nodes.shape[0]
    ndof = 2 * n
    K = np.zeros((ndof, ndof), dtype=float)
    elem_data = []

    # TODO: loop over elements, compute ke, scatter-add into K
    # DOF map for element (i,j): [2*i, 2*i+1, 2*j, 2*j+1]
    raise NotImplementedError


def apply_boundary_conditions_by_partition(K, f, fixed_dofs):
    """
    Apply essential BCs using partitioning:
        [Kff Kfc][uf] = [ff]
        [Kcf Kcc][uc]   [fc]
    with uc = 0 for fixed dofs.

    Parameters
    ----------
    K : (ndof, ndof) ndarray
    f : (ndof,) ndarray
    fixed_dofs : iterable of int
        DOFs constrained to zero.

    Returns
    -------
    free_dofs : (nfree,) ndarray
    Kff : (nfree,nfree) ndarray
    ff : (nfree,) ndarray
    """
    ndof = K.shape[0]
    fixed_dofs = np.array(sorted(set(fixed_dofs)), dtype=int)

    # TODO: construct free_dofs and return Kff and ff
    # Hint: free = all dofs not in fixed
    raise NotImplementedError


def solve_truss(nodes, elements, loads, fixed_dofs):
    """
    Solve the truss for displacements and reactions.

    Parameters
    ----------
    nodes : (n,2) ndarray
    elements : list of dict
    loads : dict {dof_index: value}
        Nodal load vector entries (forces). Units consistent with E/A/coords.
    fixed_dofs : list[int]
        Constrained DOFs (zero displacement).

    Returns
    -------
    u : (2n,) ndarray
        Global displacement vector.
    r : (2n,) ndarray
        Global reaction vector (including zeros at free DOFs).
    K : (2n,2n) ndarray
        Global stiffness.
    f : (2n,) ndarray
        Global load vector.
    elem_data : list of dict
        Cached element data.
    """
    K, elem_data = assemble_global_stiffness(nodes, elements)

    ndof = K.shape[0]
    f = np.zeros(ndof, dtype=float)
    for dof, val in loads.items():
        f[dof] += float(val)

    # Partition
    free_dofs, Kff, ff = apply_boundary_conditions_by_partition(K, f, fixed_dofs)

    # Solve
    u = np.zeros(ndof, dtype=float)
    # TODO: solve Kff * uf = ff and write into u[free_dofs]
    raise NotImplementedError

    # Reactions: r = K u - f
    r = K @ u - f
    return u, r, K, f, elem_data


def recover_element_axial_forces(nodes, elements, u, elem_data=None):
    """
    Recover axial force (tension positive) for each element.

    Approach:
    - For a 2D truss element, axial deformation = [ -c -s c s ] * ue
      where ue = [ui_x, ui_y, uj_x, uj_y]
    - axial force N = (EA/L) * axial_deformation

    Returns
    -------
    N : (nelem,) ndarray
        Axial forces for each element (tension positive).
    stress : (nelem,) ndarray
        Axial stress = N / A
    """
    ne = len(elements)
    N = np.zeros(ne, dtype=float)
    stress = np.zeros(ne, dtype=float)

    # TODO: loop over elements and compute N and stress
    # Use elem_data if provided; otherwise recompute L,c,s
    raise NotImplementedError


# Part B — Verification utilities (provided)

These checks help you *test* your solver:

- Symmetry of **K**
- Rigid body mode / singularity detection (rank)
- Global equilibrium: `f + r ≈ 0`
- Energy consistency: `U = 1/2 u^T K u` and `W = u^T f` (for linear system with correct BCs)


In [ ]:
def check_symmetry(K, tol=1e-9):
    return np.allclose(K, K.T, atol=tol, rtol=0)

def matrix_rank(K, tol=1e-10):
    s = np.linalg.svd(K, compute_uv=False)
    return int(np.sum(s > tol))

def equilibrium_check(f, r, tol=1e-6):
    residual = f + r
    return np.linalg.norm(residual, ord=np.inf), residual

def energy_check(K, u, f):
    U = 0.5 * u @ (K @ u)
    W = u @ f
    return U, W, abs(U - W)


# Part C — Benchmark problems (provided)

Run these after completing the TODOs in Part A.

---
## Benchmark 1: Two-bar truss
- Node 0 pinned (ux=uy=0)
- Node 1 roller (uy=0, ux free)
- Node 2 loaded downward Fy
- Elements: (0–2), (1–2)


In [ ]:
def benchmark_1():
    L = 2.0
    H = 1.5
    E = 200e9
    A = 3.0e-4
    Fy = -50e3  # N

    nodes = np.array([
        [0.0, 0.0],     # 0
        [L,   0.0],     # 1
        [L/2, H],       # 2
    ], dtype=float)

    elements = [
        {"i": 0, "j": 2, "E": E, "A": A},
        {"i": 1, "j": 2, "E": E, "A": A},
    ]

    loads = {2*2 + 1: Fy}  # node 2, Fy

    fixed_dofs = [0, 1, 2*1 + 1]  # ux0, uy0, uy1
    return nodes, elements, loads, fixed_dofs


## Benchmark 2: Triangle truss (3 bars)
- Elements: (0–1), (0–2), (1–2)


In [ ]:
def benchmark_2():
    L = 3.0
    H = 2.0
    E = 70e9
    A = 5.0e-4
    Fy = -25e3

    nodes = np.array([
        [0.0, 0.0],   # 0
        [L,   0.0],   # 1
        [L/2, H],     # 2
    ], dtype=float)

    elements = [
        {"i": 0, "j": 1, "E": E, "A": A},
        {"i": 0, "j": 2, "E": E, "A": A},
        {"i": 1, "j": 2, "E": E, "A": A},
    ]

    loads = {2*2 + 1: Fy}
    fixed_dofs = [0, 1, 2*1 + 1]  # pin at 0, roller y at 1
    return nodes, elements, loads, fixed_dofs


## Benchmark 3: 10-bar planar truss (standard test)

Nodes (m):
- 0:(0,0), 1:(1,0), 2:(2,0)
- 3:(0,1), 4:(1,1), 5:(2,1)

Elements (10):
- bottom: (0–1),(1–2)
- top:    (3–4),(4–5)
- verticals: (0–3),(1–4),(2–5)
- diagonals: (0–4),(1–3),(1–5)

Supports:
- Node 0 pinned (ux=uy=0)
- Node 2 roller (uy=0)

Load:
- Node 4: Fy downward


In [ ]:
def benchmark_3():
    E = 200e9
    A = 2.5e-4
    Fy = -100e3

    nodes = np.array([
        [0.0, 0.0],  # 0
        [1.0, 0.0],  # 1
        [2.0, 0.0],  # 2
        [0.0, 1.0],  # 3
        [1.0, 1.0],  # 4
        [2.0, 1.0],  # 5
    ], dtype=float)

    conn = [
        (0,1),(1,2),
        (3,4),(4,5),
        (0,3),(1,4),(2,5),
        (0,4),(1,3),(1,5),
    ]
    elements = [{"i": i, "j": j, "E": E, "A": A} for (i,j) in conn]

    loads = {2*4 + 1: Fy}
    fixed_dofs = [0, 1, 2*2 + 1]  # node0 pin, node2 roller-y
    return nodes, elements, loads, fixed_dofs


# Part D — Run & test harness (provided)

After finishing TODOs, run the cases below.


In [ ]:
def run_case(case_fn, name="Case"):
    nodes, elements, loads, fixed_dofs = case_fn()
    print(f"\n=== {name} ===")
    u, r, K, f, elem_data = solve_truss(nodes, elements, loads, fixed_dofs)
    N, stress = recover_element_axial_forces(nodes, elements, u, elem_data)

    print("K symmetric:", check_symmetry(K))
    eq_inf, eq_vec = equilibrium_check(f, r)
    print("Equilibrium ||f+r||_inf:", eq_inf)
    U, W, diff = energy_check(K, u, f)
    print("Energy U:", U, " W:", W, " |U-W|:", diff)

    n = nodes.shape[0]
    u_nodes = u.reshape(n, 2)
    print("\nDisplacements per node [ux, uy]:")
    for i in range(n):
        print(f"  node {i}: {u_nodes[i]}")

    print("\nReactions per node [Rx, Ry] (zeros at free DOFs):")
    r_nodes = r.reshape(n, 2)
    for i in range(n):
        print(f"  node {i}: {r_nodes[i]}")

    print("\nElement axial forces N (tension +) and stress:")
    for e, el in enumerate(elements):
        print(f"  e{e:02d} ({el['i']}-{el['j']}): N={N[e]: .3f}  stress={stress[e]: .3e}")

    return nodes, elements, u, r, K, f, N, stress


In [ ]:
# Uncomment these after completing TODOs:
# run_case(benchmark_1, "Benchmark 1: Two-bar truss")
# run_case(benchmark_2, "Benchmark 2: Triangle truss")
# run_case(benchmark_3, "Benchmark 3: 10-bar truss")


# Part E — Required questions (answer in markdown)

1. **Assembly sanity:** For Benchmark 3, what is the size of K? What is its rank *before* applying boundary conditions?  
   Explain what the rank indicates about rigid body modes.

2. **Equilibrium:** For each benchmark, report `||f+r||_inf`.  
   What residual level is acceptable, and what primarily controls it?

3. **Energy consistency:** Report `U`, `W`, and `|U-W|` for each benchmark.  
   Why should they match (approximately) for linear trusses with correct BC handling?

4. **Signs:** Explain why axial force tension is positive in your implementation.  
   How does your sign convention relate to the axial deformation calculation?

5. **Sensitivity:** In Benchmark 1, double **A** for both members.  
   What happens to displacements? What happens to member forces? Why?


# Part F — (Optional) Plotting helper (provided)

Use a scale factor so you can see deformations.


In [ ]:
def plot_truss(nodes, elements, u=None, scale=1.0, title="Truss"):
    import matplotlib.pyplot as plt

    nodes = np.asarray(nodes, dtype=float)
    fig, ax = plt.subplots()
    ax.set_aspect("equal", adjustable="box")

    # Undeformed
    for el in elements:
        i, j = el["i"], el["j"]
        xi, yi = nodes[i]
        xj, yj = nodes[j]
        ax.plot([xi, xj], [yi, yj], linewidth=1.0)

    if u is not None:
        u = np.asarray(u, dtype=float).reshape(nodes.shape[0], 2)
        dnodes = nodes + scale * u
        for el in elements:
            i, j = el["i"], el["j"]
            xi, yi = dnodes[i]
            xj, yj = dnodes[j]
            ax.plot([xi, xj], [yi, yj], linewidth=2.0)

    ax.set_title(title)
    ax.grid(True)
    plt.show()

# Example after solving:
# nodes, elements, u, r, K, f, N, stress = run_case(benchmark_3, "Benchmark 3")
# plot_truss(nodes, elements, u=None, title="Benchmark 3 - Undeformed")
# plot_truss(nodes, elements, u=u, scale=2000, title="Benchmark 3 - Deformed (scaled)")
